In [12]:
from openeye.oechem import *
from openeye.oegraphsim import *
import os,sys
original_file = "/Users/jfeng1/Datasets/ADME_COMPOUNDS_Selection/biogen_diverse_0.16.sdf"
resource_file = "/Users/jfeng1/Datasets/ADME_COMPOUNDS_Selection/biogen_chiral_pure_90uL_0.1mg.sdf"


In [13]:
ifs = oemolistream()
ifs.open(resource_file)
mol = OEGraphMol()
mols = []
while OEReadMolecule(ifs,mol):
    mols.append(OEGraphMol(mol))
ifs.close()
print len(mols)

237271


In [14]:
fingerprint_db = OEFPDatabase(OEFPType_Circular)
emptyfp = OEFingerPrint()
emptyfp.SetFPTypeBase(fingerprint_db.GetFPTypeBase())

for idx,mol in enumerate(mols):
    fingerprint_db.AddFP(mol)


In [ ]:

output_file = "/Users/jfeng1/Datasets/ADME_COMPOUNDS_Selection/neighbors.sdf"

ofs = oemolostream()
ofs.open(output_file)

ifs = oemolistream()
ifs.open(original_file)
mol = OEGraphMol()
molCount = 0
while OEReadMolecule(ifs,mol):
    cluster = oechem.OEGetSDData(mol,"cluster_no")
    molCount += 1
    OEWriteMolecule(ofs,mol)
    fp = OEFingerPrint()
    OEMakeFP(fp,mol,OEFPType_Circular)
    scores = fingerprint_db.GetSortedScores(fp, 10)
    nbr_count = 0
    nbr_max = 2
    for s in scores:
        if nbr_count>=2:
            break
        if s.GetScore() != 1.0:
            nbr_count += 1
            nbr_mol = mols[s.GetIdx()]
            oechem.OESetSDData(nbr_mol,"cluster_no",cluster)
            oechem.OESetSDData(nbr_mol,"cluster_nbr",str(nbr_count))
            OEWriteMolecule(ofs, nbr_mol)
ifs.close()

ofs.close()


NotImplementedError: Wrong number or type of arguments for overloaded function 'OESetSDData'.
  Possible C/C++ prototypes are:
    OEChem::OESetSDData(OEChem::OEMolBase &,std::string const &,std::string const &)
    OEChem::OESetSDData(OEChem::OEMolBase &,OEChem::OESDDataPair const &)
